In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_excel('../data/2018.xlsx')

In [3]:
!cat ../data/notes.txt



















































































Betting odds for matches generally represent the most recent before play starts, as reported by oddsportal.com and the individual bookmakers.

In [4]:
df.keys()

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts',
       'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets',
       'Lsets', 'Comment', 'B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW',
       'PSL', 'MaxW', 'MaxL', 'AvgW', 'AvgL'],
      dtype='object')

In [5]:
X = df[['Winner', 'Loser', 'WRank', 'LRank', 'Court', 'WPts', 'LPts', 'Date']].copy()

In [6]:
def get_win_loss(row, player):
    if row['Winner'] == player:
        return True
    return False

def get_ranks(row, player):
    if row['Winner'] == player:
        return row['WRank'], row['LRank']
    return row['LRank'], row['WRank']

def is_winner(row, player):
    return row['Winner'] == player

In [7]:
players = unique(df[['Winner', 'Loser']])
# form = pd.DataFrame(columns=['Winner', 'Loser', 'WRank', 'LRank', 'Court', 'WPts', 'LPts', 'Date',
#        'A', 'B', 'C', 'D', 'E'])
form = []

t = []

keys = X.keys()
for player in players:
    matches = X.loc[bitwise_or(X['Winner'] == player, X['Loser'] == player)]
    if len(matches) < 5:
        continue
    srtd = matches.sort_values('Date')
    history = []
    courtHistory = []
    opponentRank = []
    currentRank = []
    
    counter = 0
    for idx, row in srtd.iterrows():
        if counter < 5:
            history.append(get_win_loss(row, player))
            courtHistory.append(row['Court'])
            persr, oppr = get_ranks(row, player)
            opponentRank.append(oppr)
            currentRank.append(persr)
            counter += 1
            continue
        
        histFrame = pd.DataFrame([history], columns=['A', 'B', 'C', 'D', 'E'])
        courtFrame = pd.DataFrame([courtHistory], columns=['AC', 'BC', 'CC', 'DC', 'EC'])
        opponentFrame = pd.DataFrame([opponentRank], columns=['AR', 'BR', 'CR', 'DR', 'ER'])
        currentFrame = pd.DataFrame([currentRank], columns=['APR', 'BPR', 'CPR', 'DPR', 'EPR'])
        
        rowFrame = pd.DataFrame(array([row]), columns=keys)
        ctRow = pd.concat((rowFrame, histFrame, courtFrame, opponentFrame, currentFrame), axis=1)
        
        form.append(ctRow.iloc[0])
        t.append(is_winner(row, player))
        
        history.pop(0), courtHistory.pop(0), opponentRank.pop(0), currentRank.pop(0)
        history.append(get_win_loss(row, player)), courtHistory.append(row['Court'])
        persr, oppr = get_ranks(row, player)
        opponentRank.append(persr), currentRank.append(oppr)
 

In [8]:
form = pd.DataFrame(form)

In [9]:
X = form[[
    'WRank', 'LRank', 'Court', 'WPts', 'LPts',
    'A', 'B', 'C', 'D', 'E',
    'AC', 'BC', 'CC', 'DC', 'EC',
    'AR', 'BR', 'CR', 'DR', 'ER',
    'APR', 'BPR', 'CPR', 'DPR', 'EPR'
]]

In [10]:
t[0], count_nonzero([t[0]])

(True, 1)

In [11]:
print(F'There are {len(t)} samples, with {len(X.keys())} input features.')
print(F'There are {len(t) - count_nonzero(t)} losses and {count_nonzero(t)} wins.')

There are 4194 samples, with 25 input features.
There are 2027 losses and 2167 wins.


In [12]:
s = pd.Series(t)
s.to_csv('../data/targets.csv')
X.to_csv('../data/inputs.csv')

/anaconda3/envs/eco/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
